In [2]:
import numpy as np
import pandas as pd


In [8]:
import re

In [9]:
def feature_engineering(df, sparse=0): 
    
    # Comment length
    df['length'] = df.comment_text.apply(lambda x: len(x))
    

    # Capitalization percentage
    def pct_caps(s):
        return sum([1 for c in s if c.isupper()]) / (sum(([1 for c in s if c.isalpha()])) + 1)
    df['caps'] = df.comment_text.apply(lambda x: pct_caps(x))

    # Mean Word length 
    def word_length(s):
        s = s.split(' ')
        return np.mean([len(w) for w in s if w.isalpha()])
    df['word_length'] = df.comment_text.apply(lambda x: word_length(x))

    # Average number of exclamation points 
    df['exclamation'] = df.comment_text.apply(lambda s: len([c for c in s if c == '!']))

    # Average number of question marks 
    df['question'] = df.comment_text.apply(lambda s: len([c for c in s if c == '?']))
    
    # Normalize
    for label in ['length', 'caps', 'word_length', 'question', 'exclamation']:
        minimum = df[label].min()
        diff = df[label].max() - minimum
        df[label] = df[label].apply(lambda x: (x-minimum) / (diff))

    # Strip IP Addresses
    ip = re.compile('(([2][5][0-5]\.)|([2][0-4][0-9]\.)|([0-1]?[0-9]?[0-9]\.)){3}'
                    +'(([2][5][0-5])|([2][0-4][0-9])|([0-1]?[0-9]?[0-9]))')
    def strip_ip(s, ip):
        try:
            found = ip.search(s)
            return s.replace(found.group(), ' ')
        except:
            return s

    df.comment_text = df.comment_text.apply(lambda x: strip_ip(x, ip))
    
    return df

def merge_features(comment_text, data, engineered_features):
    new_features = sparse.csr_matrix(df[engineered_features].values)
    if np.isnan(new_features.data).any():
        new_features.data = np.nan_to_num(new_features.data)
    return sparse.hstack([comment_text, new_features])

In [10]:

df = pd.read_csv('./fypenv/dataset/train.csv')
targets = list(df.columns[2:])
df_targets = df[targets].copy()

df_sub = pd.read_csv('./fypenv/dataset/test.csv', dtype={'id': object}, na_filter=False)

submission = pd.DataFrame()
submission['id'] = df_sub.id.copy()

# Feature Engineering
df = feature_engineering(df)
df_sub = feature_engineering(df_sub)

print('Training labels:')
print(list(df_targets.columns))
print(df_targets.shape)

print('\nTraining data')
df.drop(list(df_targets.columns), inplace=True, axis=1)
df.drop('id', inplace=True, axis=1)
print(list(df.columns))
print(df.shape)


print('\nSubmission data')
df_sub.drop('id', inplace=True, axis=1)
print(list(df_sub.columns))
print(df_sub.shape)

toxic_rows = df_targets.sum(axis=1)
toxic_rows = (toxic_rows > 0)
targets.append('any_label')
df_targets['any_label'] = toxic_rows.astype(int)

new_features = list(df.columns[1:])
print(new_features)

# from sklearn.model_selection import train_test_split
# df, holdout, df_targets, holdout_targets = train_test_split(df, df_targets, test_size=0.2, random_state=seed)

c:\Fyp\fypenv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Fyp\fypenv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Fyp\fypenv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Fyp\fypenv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Training labels:
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
(159571, 6)

Training data
['comment_text', 'length', 'caps', 'word_length', 'exclamation', 'question']
(159571, 6)

Submission data
['comment_text', 'length', 'caps', 'word_length', 'exclamation', 'question']
(153164, 6)
['length', 'caps', 'word_length', 'exclamation', 'question']


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
comment_vector = TfidfVectorizer(max_features=10000, analyzer='word', #ngram_range=(2, 6), 
                                 stop_words='english')
training_comments = comment_vector.fit_transform(df.comment_text)
# holdout_comments = comment_vector.transform(holdout.comment_text)
submission_comments = comment_vector.transform(df_sub.comment_text)

In [21]:
training_comments



<159571x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3495379 stored elements in Compressed Sparse Row format>

In [20]:
submission_comments

<153164x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 2929468 stored elements in Compressed Sparse Row format>